# Magenta RT: Gerçek Zamanlı Ses Girişiyle Koşullandırma

Bu Colab not defteri, Magenta RT modelini mikrofonunuzdan gelen gerçek zamanlı ses girişiyle koşullandırmanıza olanak tanır. Yakalanan ses, müziğin stilini etkilemek için kullanılacaktır.

**Önemli Notlar:**
*   Bu not defterini çalıştırmak için bir TPU çalışma zamanına (`Runtime > Change Runtime Type > TPU`) ve tarayıcınızda mikrofon erişimine izin vermeniz gerekecektir.
*   Gerçek zamanlı ses işleme ve model çıkarımı nedeniyle bir miktar gecikme yaşanabilir.
*   En iyi sonuçlar için sessiz bir ortamda net bir ses girişi sağlayın.

## Adım 1: Kurulum ve Gerekli Bileşenlerin Başlatılması

Bu ilk adımda, Magenta RT kütüphanesini ve bağımlılıklarını yükleyip, önceden eğitilmiş Magenta RT modelini ve ilişkili kodekleri başlatacağız. Modelin hemen kullanıma hazır olması için `lazy=False` ile yüklenecektir.

In [ ]:
# @title **Bu hücreyi çalıştırın** bağımlılıkları yüklemek için (~5 dakika)
# @markdown TPU çalışma zamanı kullandığınızdan emin olun (`Runtime > Change Runtime Type`).

# @markdown Colab, oturumu yeniden başlatmanızı isteyebilir. **Yeniden başlatmadan önce hücrenin çalışmasının bitmesini bekleyin!** Oturum yeniden başlatıldıktan sonra bir sonraki hücreye geçin.

print("Magenta RealTime deposu klonlanıyor...")
!git clone https://github.com/magenta/magenta-realtime.git

print("Gerekli TensorFlow sürümleri ayarlanıyor...")
_all_tf_rt = 'tensorflow tf-nightly tensorflow-cpu tf-nightly-cpu tensorflow-tpu tf-nightly-tpu tensorflow-hub tf-hub-nightly tensorflow-text tensorflow-text-nightly'
_nightly_tf_rt = 'tf-nightly tensorflow-text-nightly tf-hub-nightly'

print("Kütüphane ve bağımlılıklar yükleniyor (TPU için)...")
!pip install -e magenta-realtime/[tpu] && pip uninstall -y {_all_tf_rt} && pip install {_nightly_tf_rt}

print("Kurulum tamamlandı. Eğer Colab tarafından istendiyse, şimdi 'Runtime > Restart Session' menüsünden oturumu yeniden başlatın ve bir sonraki hücreyle devam edin.")

In [ ]:
# @title **Oturumu yeniden başlattıysanız bu hücreyi çalıştırın** modeli ve diğer bileşenleri başlatmak için.
# @markdown Modelin yüklenmesi biraz zaman alabilir (~5 dakika).

import os
import numpy as np
import tensorflow as tf

print(f"TensorFlow sürümü: {tf.__version__}")
print("Gerekli modüller yükleniyor...")
from magenta_rt import system
from magenta_rt import audio
from magenta_rt import musiccoca
from magenta_rt import spectrostream
from magenta_rt.colab import utils as magenta_colab_utils # İsim çakışmasını önlemek için
from magenta_rt.colab import widgets as magenta_colab_widgets 

from google.colab import output 
from IPython.display import display, Javascript 
import IPython.display as ipd 
import ipywidgets as ipw 

import librosa
import time
import threading 
import queue 
import tempfile # Geçici dosyalar için
import subprocess # FFMPEG için
import io # BytesIO için

print("Magenta RT modeli yükleniyor (lazy=False)...")
MODEL_TAG_FOR_REALTIME_INPUT = "large" #@param ["base", "large"]
MRT_REALTIME_INPUT_MODEL = None # Model nesnesini tutacak global değişken
try:
  MRT_REALTIME_INPUT_MODEL = system.MagentaRT(
      tag=MODEL_TAG_FOR_REALTIME_INPUT, 
      device="tpu:v2-8", 
      skip_cache=True, 
      lazy=False # Gerçek zamanlı etkileşim için modeli hemen yükle
  )
  print(f"Magenta RT '{MODEL_TAG_FOR_REALTIME_INPUT}' modeli başarıyla yüklendi.")
except Exception as e_init:
  print(f"Model yüklenirken bir hata oluştu: {e_init}")
  print("Lütfen çalışma zamanının TPU olarak ayarlandığından ve önceki hücrenin başarıyla çalıştırıldığından emin olun.")

print("Başlatma ve hazırlık adımı tamamlandı.")

## Adım 2: Gerçek Zamanlı Ses Yakalama ve Müzik Üretim Arayüzü

Bu adım, tarayıcınızın mikrofonundan periyodik olarak ses yakalamak, yakalanan sesi işleyerek stil gömmesi oluşturmak ve bu stil gömmesini kullanarak Magenta RT ile sürekli müzik üretmek için gerekli JavaScript ve Python kodlarını ayarlar.

**Önemli Notlar:**
*   Aşağıdaki ilk JavaScript kod hücresini çalıştırdıktan sonra, bir sonraki Python hücreleri çalıştırıldığında tarayıcınız **mikrofon erişimi için izin isteyebilir.** Lütfen "İzin Ver" (Allow) seçeneğini tıklayın.
*   **FFMPEG Kontrolü:** Bu not defteri, tarayıcıdan gelen sesi (genellikle webm veya ogg formatında) işlemek için FFMPEG'e ihtiyaç duyar. Python hücreleri FFMPEG'in kurulu olup olmadığını kontrol edecektir. Eğer kurulu değilse, bir uyarı mesajı göreceksiniz ve arayüz düzgün çalışmayacaktır. Colab'da FFMPEG genellikle `!apt update && apt install -y ffmpeg` komutuyla kurulabilir (yeni bir hücrede çalıştırıp ardından "Runtime > Restart session" yapmanız gerekebilir).
*   Ses yakalama ve işleme süreci, özellikle format dönüşümü nedeniyle bir miktar gecikmeye sahip olabilir. 
*   En iyi sonuçlar için, mümkün olduğunca sessiz bir ortamda ve net bir ses girişiyle (örneğin, doğrudan mikrofona konuşarak veya bir enstrüman çalarak) kullanın.

In [ ]:
# @title JavaScript ile Periyodik Ses Yakalama Fonksiyonları (vForSure)

RT_FORSURE_REC_SEC = 1.8 #@param {type:"number", min:0.5, max:3, step:0.1}
#@markdown Her bir ses yakalama periyodunun saniye cinsinden uzunluğu.

RT_FORSURE_JS_CODE = f"""
// JS Globals
let rtForSureMediaRecorder;
let rtForSureRecordedChunks = [];
let rtForSureMediaStream;
let rtForSureAudioContext;
let rtForSureIsCapturing = false;
let rtForSureTimeoutId = null;
const RT_FORSURE_STATUS_ID = 'rt_forsure_status_js_id'; // Benzersiz ID

function rtForSureUpdateStatus(msg) {{
  const el = document.getElementById(RT_FORSURE_STATUS_ID);
  if (el) el.innerText = msg;
  console.log('RT_FORSURE JS: ' + msg);
}}

if (!navigator.mediaDevices || !navigator.mediaDevices.getUserMedia) {{
  rtForSureUpdateStatus('Hata: Mikrofon API desteklenmiyor.');
}}

async function rtForSureStartCapture(durationSec) {{
  if (!navigator.mediaDevices || !navigator.mediaDevices.getUserMedia) {{
    rtForSureUpdateStatus('Hata: Mikrofon API desteklenmiyor.');
    return Promise.reject('Mikrofon API yok.');
  }}
  rtForSureIsCapturing = true;
  rtForSureUpdateStatus('Mikrofon hazırlanıyor...');

  async function captureCycleInternalJsForSure() {{
    if (!rtForSureIsCapturing) {{
      await rtForSureReleaseResourcesJs(); return;
    }}
    try {{
      await rtForSureReleaseResourcesJs();
      rtForSureMediaStream = await navigator.mediaDevices.getUserMedia({{ audio: {{echoCancellation: true, noiseSuppression: true}}, video: false }});
      rtForSureAudioContext = new (window.AudioContext || window.webkitAudioContext)();
      const optsJsForSure = {{ mimeType: 'audio/webm;codecs=opus' }};
      if (!MediaRecorder.isTypeSupported(optsJsForSure.mimeType)) {{
          optsJsForSure.mimeType = 'audio/ogg;codecs=opus'; 
          if (!MediaRecorder.isTypeSupported(optsJsForSure.mimeType)) delete optsJsForSure.mimeType;
      }}
      rtForSureMediaRecorder = new MediaRecorder(rtForSureMediaStream, optsJsForSure);
      rtForSureRecordedChunks = [];
      rtForSureMediaRecorder.ondataavailable = e => {{ if (e.data.size > 0) rtForSureRecordedChunks.push(e.data); }};
      rtForSureMediaRecorder.onstop = async () => {{
        if (rtForSureRecordedChunks.length === 0 && rtForSureIsCapturing) {{
            google.colab.kernel.invokeFunction('notebook.py_rt_forsure_audio_handler_js', ['', null], {{}});
            return;
        }}
        if (rtForSureRecordedChunks.length > 0){ 
            const blobJsForSure = new Blob(rtForSureRecordedChunks, {{ type: rtForSureMediaRecorder.mimeType }});
            const bufferJsForSure = await blobJsForSure.arrayBuffer();
            let binStrJsForSure = '';
            const bytesArrJsForSure = new Uint8Array(bufferJsForSure);
            for (let i = 0; i < bytesArrJsForSure.byteLength; i++) binStrJsForSure += String.fromCharCode(bytesArrJsForSure[i]);
            const base64JsForSure = window.btoa(binStrJsForSure);
            google.colab.kernel.invokeFunction('notebook.py_rt_forsure_audio_handler_js', [base64JsForSure, rtForSureMediaRecorder.mimeType], {{}});
            rtForSureRecordedChunks = [];
        }}
      }};
      rtForSureMediaRecorder.onerror = (errEvtJsForSure) => {{
        rtForSureUpdateStatus('MediaRecorder Hatası: ' + errEvtJsForSure.error.name);
        rtForSureErrorStopJs();
      }};
      rtForSureMediaRecorder.start();
      rtForSureUpdateStatus('🎙️ Mikrofon ' + durationSec + 'sn dinleniyor... (MIME: ' + rtForSureMediaRecorder.mimeType + ')');
      if (rtForSureTimeoutId) clearTimeout(rtForSureTimeoutId);
      rtForSureTimeoutId = setTimeout(() => {{
          if (rtForSureMediaRecorder && rtForSureMediaRecorder.state === 'recording') rtForSureMediaRecorder.stop();
      }}, durationSec * 1000);
    }} catch (catchedErrJsForSure) {{
      rtForSureUpdateStatus('Mikrofon/Yakalama Hatası: ' + catchedErrJsForSure.name + '. İzinleri kontrol edin.');
      rtForSureErrorStopJs();
    }}
  }}
  captureCycleInternalJsForSure();
  return Promise.resolve('JS Periyodik yakalama başlatıldı.');
}};

async function rtForSureReleaseResourcesJs() {{
    if (rtForSureTimeoutId) clearTimeout(rtForSureTimeoutId); rtForSureTimeoutId = null;
    if (rtForSureMediaRecorder && rtForSureMediaRecorder.state === 'recording') {{
        rtForSureMediaRecorder.onstop = null; rtForSureMediaRecorder.stop();
    }} rtForSureMediaRecorder = null;
    if (rtForSureMediaStream) rtForSureMediaStream.getTracks().forEach(trJsForSure => {{ trJsForSure.stop(); }}); rtForSureMediaStream = null;
    if (rtForSureAudioContext && rtForSureAudioContext.state !== 'closed') {{
        await rtForSureAudioContext.close().catch(e => {{}}); 
    }} rtForSureAudioContext = null;
}}

async function rtForSureStopJsCapture() {{
    rtForSureIsCapturing = false;
    await rtForSureReleaseResourcesJs();
    rtForSureUpdateStatus('Mikrofon ve ses yakalama durduruldu.');
}}

function rtForSureErrorStopJs(){
    rtForSureIsCapturing = false; 
    rtForSureReleaseResourcesJs(); 
    google.colab.kernel.invokeFunction('notebook.py_rt_forsure_js_error_handler', [], {{}});
}
"""
display(Javascript(RT_FORSURE_JS_CODE))
print(f"Gerçek zamanlı ses yakalama için JS fonksiyonları (vForSure) tanımlandı. Periyot: {RT_FORSURE_REC_SEC}sn.")

In [ ]:
# @title Python Tarafı: Ses İşleyici, Kontroller ve Müzik Üretim Döngüsü (vForSure)
import base64, tempfile, subprocess, io, time, threading, queue, os
import numpy as np
import librosa
from magenta_rt import audio 
from google.colab import output 
import IPython.display as ipd
import ipywidgets as ipw

# Python Globals
PY_RT_FORSURE_Q = queue.Queue(maxsize=2)
PY_RT_FORSURE_GENERATING = False 
PY_RT_FORSURE_THREAD = None
PY_RT_FORSURE_STOP_EVT = threading.Event()

def py_rt_forsure_js_err_handler_py():
    print(f"RT_FORSURE Py: JS Hata Sinyali! Durduruluyor...")
    py_rt_forsure_stop_btn_handler_py(None) 
    return "RT_FORSURE Py: JS Hatası işlendi."
output.register_ लेकिन('notebook.py_rt_forsure_js_error_handler', py_rt_forsure_js_err_handler_py)

def py_rt_forsure_audio_handler_js_py(b64_data_py_forsure, mime_type_js_py_forsure):
    global PY_RT_FORSURE_GENERATING, PY_RT_FORSURE_STOP_EVT
    processed_py_ok_flag_forsure = False
    try:
        if not b64_data_py_forsure:
            pass 
        else:
            decoded_py_bytes_audio_forsure = base64.b64decode(b64_data_py_forsure)
            suffix_py_str_audio_forsure = mime_type_js_py_forsure.split('/')[-1].split(';')[0] if mime_type_js_py_forsure else 'webm'
            with tempfile.NamedTemporaryFile(suffix=f'.{suffix_py_str_audio_forsure}', delete=False) as tmp_in_f_py_audio_forsure:
                tmp_in_f_py_audio_forsure.write(decoded_py_bytes_audio_forsure)
                raw_py_path_audio_forsure = tmp_in_f_py_audio_forsure.name
            wav_py_out_path_audio_forsure = tempfile.mktemp(suffix='.wav')
            sr_py_target_audio_forsure = MRT_REALTIME_INPUT_MODEL.style_model.config.sample_rate # MRT_realtime_input yerine global değişkeni kullan
            ffmpeg_py_list_cmd_forsure = ['ffmpeg', '-y', '-i', raw_py_path_audio_forsure, '-ac', '1', '-ar', str(sr_py_target_audio_forsure), '-vn', '-c:a', 'pcm_s16le', wav_py_out_path_audio_forsure]
            ffmpeg_py_res_audio_forsure = subprocess.run(ffmpeg_py_list_cmd_forsure, capture_output=True, text=True, check=False)
            if os.path.exists(raw_py_path_audio_forsure): os.remove(raw_py_path_audio_forsure)
            if ffmpeg_py_res_audio_forsure.returncode == 0:
                if os.path.exists(wav_py_out_path_audio_forsure):
                    with open(wav_py_out_path_audio_forsure, 'rb') as f_wav_py_in_audio_forsure:
                        wav_bytes_py_final_audio_forsure = f_wav_py_in_audio_forsure.read()
                    if PY_RT_FORSURE_Q.full(): PY_RT_FORSURE_Q.get_nowait()
                    PY_RT_FORSURE_Q.put(wav_bytes_py_final_audio_forsure)
                    processed_py_ok_flag_forsure = True
            else: print(f"RT_FORSURE Py: FFMPEG Hata ({ffmpeg_py_res_audio_forsure.returncode}): {ffmpeg_py_res_audio_forsure.stderr}")
            if os.path.exists(wav_py_out_path_audio_forsure): os.remove(wav_py_out_path_audio_forsure)
    except Exception as exc_py_audio_proc_forsure:
        print(f"RT_FORSURE Py: Ses işleme hatası: {exc_py_audio_proc_forsure}")
    finally:
        if PY_RT_FORSURE_GENERATING and not PY_RT_FORSURE_STOP_EVT.is_set():
            py_rt_forsure_trigger_js_capture_py() 
    return f"RT_FORSURE Py: Veri işlendi (OK: {processed_py_ok_flag_forsure})."

output.register_ लेकिन('notebook.py_rt_forsure_audio_handler_js', py_rt_forsure_audio_handler_js_py)

rt_forsure_start_btn = ipw.Button(description="🎙️ Canlı Stil Başlat", disabled=not ('MRT_REALTIME_INPUT_MODEL' in globals() and MRT_REALTIME_INPUT_MODEL is not None))
rt_forsure_stop_btn = ipw.Button(description="🛑 Durdur", disabled=True)
rt_forsure_status_html = ipw.HTML(value=f"<p id='{RT_FORSURE_JS_CODE.split('=')[1].split(';')[0].strip().replace('`', '')}'>Durum: Hazır</p>")
rt_forsure_audio_out = ipw.Output()

def py_rt_forsure_trigger_js_capture_py():
    if PY_RT_FORSURE_GENERATING and not PY_RT_FORSURE_STOP_EVT.is_set():
      display(Javascript(f'rtForSureStartCapture({RT_FORSURE_REC_SEC});'))

def py_rt_forsure_music_thread_py_func():
    global PY_RT_FORSURE_GENERATING, MRT_REALTIME_INPUT_MODEL
    mrt_state_py_forsure_internal = None
    style_emb_py_forsure_internal_current = MRT_REALTIME_INPUT_MODEL.embed_style("smooth jazz") 
    print("RT_FORSURE Py: Müzik thread'i başlatıldı.")
    while not PY_RT_FORSURE_STOP_EVT.is_set():
        new_style_py_forsure_applied_flag = False
        if not PY_RT_FORSURE_Q.empty():
            try:
                wav_q_bytes_py_forsure_internal = PY_RT_FORSURE_Q.get_nowait()
                y_mic_py_forsure_internal, sr_mic_py_forsure_internal = librosa.load(io.BytesIO(wav_q_bytes_py_forsure_internal), sr=MRT_REALTIME_INPUT_MODEL.style_model.config.sample_rate, mono=True)
                if y_mic_py_forsure_internal.size > int(MRT_REALTIME_INPUT_MODEL.style_model.config.sample_rate * 0.05):
                    mic_wf_py_forsure_internal_obj = audio.Waveform(y_mic_py_forsure_internal.astype(np.float32), sr_mic_py_forsure_internal)
                    style_emb_py_forsure_internal_current = MRT_REALTIME_INPUT_MODEL.embed_style(mic_wf_py_forsure_internal_obj)
                    new_style_py_forsure_applied_flag = True
            except Exception as exc_q_py_forsure_internal_proc:
                print(f"RT_FORSURE Py: Kuyruk WAV işleme hatası: {exc_q_py_forsure_internal_proc}")
        try:
            gen_chunk_py_forsure_obj, mrt_state_py_forsure_internal = MRT_REALTIME_INPUT_MODEL.generate_chunk(state=mrt_state_py_forsure_internal, style=style_emb_py_forsure_internal_current)
            with rt_forsure_audio_out:
                ipd.clear_output(wait=True)
                msg_py_forsure_style_source = "(🎤 Mikrofon stiliyle)" if new_style_py_forsure_applied_flag else "(Son bilinen stille)"
                print(msg_py_forsure_style_source)
                ipd.display(ipd.Audio(gen_chunk_py_forsure_obj.samples.T, rate=gen_chunk_py_forsure_obj.sample_rate, autoplay=True, normalize=False))
        except Exception as exc_gen_py_forsure_internal_proc:
            print(f"RT_FORSURE Py: Müzik üretim hatası: {exc_gen_py_forsure_internal_proc}")
            PY_RT_FORSURE_STOP_EVT.set()
            rt_forsure_status_html.value = f"<p id='{RT_FORSURE_JS_CODE.split('=')[1].split(';')[0].strip().replace('`', '')}'>Hata: {exc_gen_py_forsure_internal_proc}</p>"
            break 
        time.sleep(max(0.05, MRT_REALTIME_INPUT_MODEL.config.chunk_length - RT_FORSURE_REC_SEC + 0.1)) # Dinamik bekleme, daha fazla tolerans
    with rt_forsure_audio_out: ipd.clear_output(wait=False)
    print("RT_FORSURE Py: Müzik thread'i sonlandı.")

def py_rt_forsure_start_btn_handler_py(b):
    global PY_RT_FORSURE_GENERATING, PY_RT_FORSURE_THREAD
    if not PY_RT_FORSURE_GENERATING:
        if MRT_REALTIME_INPUT_MODEL is None:
            rt_forsure_status_html.value = f"<p id='{RT_FORSURE_JS_CODE.split('=')[1].split(';')[0].strip().replace('`', '')}'>HATA: Model yüklenmedi!</p>"
            return
        PY_RT_FORSURE_GENERATING = True
        rt_forsure_start_btn.disabled = True
        rt_forsure_stop_btn.disabled = False
        rt_forsure_status_html.value = f"<p id='{RT_FORSURE_JS_CODE.split('=')[1].split(';')[0].strip().replace('`', '')}'>Başlatılıyor...</p>"
        PY_RT_FORSURE_STOP_EVT.clear()
        while not PY_RT_FORSURE_Q.empty(): PY_RT_FORSURE_Q.get_nowait()
        PY_RT_FORSURE_THREAD = threading.Thread(target=py_rt_forsure_music_thread_py_func, daemon=True)
        PY_RT_FORSURE_THREAD.start()
        py_rt_forsure_trigger_js_capture_py()
        print("RT_FORSURE Py: Canlı stil ve müzik üretimi başlatıldı.")

def py_rt_forsure_stop_btn_handler_py(b):
    global PY_RT_FORSURE_GENERATING, PY_RT_FORSURE_THREAD
    if PY_RT_FORSURE_GENERATING or (PY_RT_FORSURE_THREAD is not None and PY_RT_FORSURE_THREAD.is_alive()):
        PY_RT_FORSURE_GENERATING = False 
        PY_RT_FORSURE_STOP_EVT.set()
        display(Javascript('rtForSureStopJsCapture();'))
        rt_forsure_status_html.value = f"<p id='{RT_FORSURE_JS_CODE.split('=')[1].split(';')[0].strip().replace('`', '')}'>Durduruluyor...</p>"
        if PY_RT_FORSURE_THREAD and PY_RT_FORSURE_THREAD.is_alive():
            PY_RT_FORSURE_THREAD.join(timeout=RT_FORSURE_REC_SEC + 2.0)
        PY_RT_FORSURE_THREAD = None 
        rt_forsure_status_html.value = f"<p id='{RT_FORSURE_JS_CODE.split('=')[1].split(';')[0].strip().replace('`', '')}'>Durduruldu.</p>"
        print("RT_FORSURE Py: Canlı stil ve müzik üretimi durduruldu.")
    else: rt_forsure_status_html.value = f"<p id='{RT_FORSURE_JS_CODE.split('=')[1].split(';')[0].strip().replace('`', '')}'>Zaten durdurulmuş.</p>"
    if 'MRT_REALTIME_INPUT_MODEL' in globals() and MRT_REALTIME_INPUT_MODEL is not None: rt_forsure_start_btn.disabled = False
    rt_forsure_stop_btn.disabled = True
    while not PY_RT_FORSURE_Q.empty():
        try: PY_RT_FORSURE_Q.get_nowait() 
        except queue.Empty: break

rt_forsure_start_btn.on_click(py_rt_forsure_start_btn_handler_py)
rt_forsure_stop_btn.on_click(py_rt_forsure_stop_btn_handler_py)

if 'MRT_REALTIME_INPUT_MODEL' in globals() and MRT_REALTIME_INPUT_MODEL is not None:
    print("Gerçek zamanlı arayüz (vForSure) hazır.")
    try:
        subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True, check=True); print("FFMPEG bulundu.")
        display(ipw.VBox([
            ipw.HTML("<b>Mikrofon ile Canlı Müzik:</b>"), 
            ipw.HBox([rt_forsure_start_btn, rt_forsure_stop_btn]), 
            rt_forsure_status_html, 
            ipw.HTML("<hr><b>Üretilen Müzik (Canlı Otomatik Çalma):</b>"), 
            rt_forsure_audio_out 
        ]))
    except (FileNotFoundError, subprocess.CalledProcessError):
        rt_forsure_start_btn.disabled = True
        rt_forsure_status_html.value = f"<p id='{RT_FORSURE_JS_CODE.split('=')[1].split(';')[0].strip().replace('`', '')}'>HATA: FFMPEG GEREKLİ! Kurulum talimatlarını kontrol edin.</p>"
        display(ipw.VBox([ipw.HTML("<b>UYARI: FFMPEG bulunamadı/çalışmıyor!</b> Bu not defteri sesi işlemek için FFMPEG gerektirir. Lütfen Colab'da yeni bir hücrede `!apt update && apt install -y ffmpeg` komutunu çalıştırıp, ardından 'Runtime > Restart session' menüsünden oturumu yeniden başlatın ve bu not defterini tekrar çalıştırın."), rt_forsure_status_html]))
else:
    rt_forsure_start_btn.disabled = True
    rt_forsure_status_html.value = f"<p id='{RT_FORSURE_JS_CODE.split('=')[1].split(';')[0].strip().replace('`', '')}'>HATA: Model yüklenemedi.</p>"
    display(ipw.VBox([ipw.HTML("<b>HATA: Model Yüklenemedi.</b> Adım 1'i kontrol edin."), ipw.HBox([rt_forsure_start_btn, rt_forsure_stop_btn]), rt_forsure_status_html ]))
